<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import numpy as np

**Helper Functions**

In [2]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1, 2) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [3]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds


**RQI Algorithm with Additive Preconditioning**

In [4]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds, conds_C, conds_desired
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [5]:
# Setup

# create a random matrix M
M = np.random.rand(100, 100)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [6]:
# Print condition number of M
svd_cond(M)

2052.8091944870075

In [7]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.620148578487346+0j)
Eigenvector: [0.09296707+0.j 0.10481125+0.j 0.09983689+0.j 0.09707312+0.j
 0.10439354+0.j 0.09794769+0.j 0.10813267+0.j 0.09284345+0.j
 0.09431399+0.j 0.10806847+0.j 0.1065417 +0.j 0.10052339+0.j
 0.10371857+0.j 0.09771615+0.j 0.09982381+0.j 0.105404  +0.j
 0.0997872 +0.j 0.10077501+0.j 0.10123154+0.j 0.09082776+0.j
 0.09291646+0.j 0.10478071+0.j 0.09539226+0.j 0.10076079+0.j
 0.10211811+0.j 0.10039803+0.j 0.10567802+0.j 0.10773907+0.j
 0.10150484+0.j 0.10974688+0.j 0.09615539+0.j 0.10295496+0.j
 0.11010318+0.j 0.09221921+0.j 0.09679558+0.j 0.09292483+0.j
 0.10536285+0.j 0.09831243+0.j 0.09759013+0.j 0.10336412+0.j
 0.09404511+0.j 0.10061831+0.j 0.09375441+0.j 0.10209773+0.j
 0.09780708+0.j 0.10253502+0.j 0.0990027 +0.j 0.10020791+0.j
 0.11025788+0.j 0.10165613+0.j 0.0968776 +0.j 0.10145509+0.j
 0.09507951+0.j 0.0995886 +0.j 0.09666881+0.j 0.10129012+0.j
 0.10075344+0.j 0.0963895 +0.j 0.09067847+0.j 0.10220834+0.j
 0.10205382+0.j 0.10148665+0.j 0.103

In [8]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.620148581414085+0j)
Eigenvector: [0.09296708+0.j 0.10481125+0.j 0.09983688+0.j 0.09707312+0.j
 0.10439354+0.j 0.09794769+0.j 0.10813267+0.j 0.09284345+0.j
 0.09431399+0.j 0.10806847+0.j 0.1065417 +0.j 0.10052338+0.j
 0.10371858+0.j 0.09771616+0.j 0.0998238 +0.j 0.10540398+0.j
 0.0997872 +0.j 0.10077501+0.j 0.10123154+0.j 0.09082776+0.j
 0.09291646+0.j 0.10478071+0.j 0.09539226+0.j 0.10076078+0.j
 0.10211811+0.j 0.10039804+0.j 0.10567801+0.j 0.10773907+0.j
 0.10150484+0.j 0.10974688+0.j 0.0961554 +0.j 0.10295496+0.j
 0.11010317+0.j 0.09221921+0.j 0.09679559+0.j 0.09292484+0.j
 0.10536284+0.j 0.09831243+0.j 0.09759014+0.j 0.10336412+0.j
 0.09404512+0.j 0.10061832+0.j 0.0937544 +0.j 0.10209774+0.j
 0.09780708+0.j 0.10253504+0.j 0.0990027 +0.j 0.10020791+0.j
 0.11025789+0.j 0.10165614+0.j 0.0968776 +0.j 0.10145509+0.j
 0.09507951+0.j 0.09958859+0.j 0.09666882+0.j 0.10129013+0.j
 0.10075344+0.j 0.09638949+0.j 0.09067848+0.j 0.10220834+0.j
 0.10205382+0.j 0.10148665+0.j 0.103

In [9]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.62014857735848+0j)
Eigenvector: [0.09296707+0.j 0.10481124+0.j 0.09983689+0.j 0.09707312+0.j
 0.10439353+0.j 0.09794769+0.j 0.10813267+0.j 0.09284344+0.j
 0.09431399+0.j 0.10806847+0.j 0.1065417 +0.j 0.10052338+0.j
 0.10371858+0.j 0.09771615+0.j 0.0998238 +0.j 0.10540399+0.j
 0.0997872 +0.j 0.10077501+0.j 0.10123154+0.j 0.09082776+0.j
 0.09291646+0.j 0.10478071+0.j 0.09539226+0.j 0.10076079+0.j
 0.10211811+0.j 0.10039803+0.j 0.10567802+0.j 0.10773907+0.j
 0.10150483+0.j 0.10974688+0.j 0.09615539+0.j 0.10295496+0.j
 0.11010317+0.j 0.0922192 +0.j 0.09679558+0.j 0.09292483+0.j
 0.10536285+0.j 0.09831243+0.j 0.09759013+0.j 0.10336412+0.j
 0.09404511+0.j 0.10061831+0.j 0.0937544 +0.j 0.10209773+0.j
 0.09780707+0.j 0.10253503+0.j 0.0990027 +0.j 0.10020791+0.j
 0.11025789+0.j 0.10165614+0.j 0.0968776 +0.j 0.10145509+0.j
 0.09507951+0.j 0.0995886 +0.j 0.09666882+0.j 0.10129013+0.j
 0.10075344+0.j 0.0963895 +0.j 0.09067848+0.j 0.10220834+0.j
 0.10205383+0.j 0.10148665+0.j 0.1039

In [10]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.620148576680755+0j)
Eigenvector: [0.09296707+0.j 0.10481125+0.j 0.09983689+0.j 0.09707313+0.j
 0.10439353+0.j 0.09794769+0.j 0.10813267+0.j 0.09284344+0.j
 0.09431399+0.j 0.10806847+0.j 0.1065417 +0.j 0.10052339+0.j
 0.10371857+0.j 0.09771615+0.j 0.0998238 +0.j 0.10540399+0.j
 0.0997872 +0.j 0.10077502+0.j 0.10123154+0.j 0.09082776+0.j
 0.09291646+0.j 0.10478071+0.j 0.09539226+0.j 0.10076079+0.j
 0.10211811+0.j 0.10039803+0.j 0.10567802+0.j 0.10773907+0.j
 0.10150483+0.j 0.10974688+0.j 0.09615539+0.j 0.10295496+0.j
 0.11010317+0.j 0.09221921+0.j 0.09679558+0.j 0.09292484+0.j
 0.10536285+0.j 0.09831243+0.j 0.09759013+0.j 0.10336412+0.j
 0.09404511+0.j 0.10061831+0.j 0.0937544 +0.j 0.10209773+0.j
 0.09780707+0.j 0.10253503+0.j 0.0990027 +0.j 0.10020791+0.j
 0.11025789+0.j 0.10165614+0.j 0.0968776 +0.j 0.10145509+0.j
 0.09507951+0.j 0.0995886 +0.j 0.09666882+0.j 0.10129013+0.j
 0.10075344+0.j 0.0963895 +0.j 0.09067848+0.j 0.10220834+0.j
 0.10205382+0.j 0.10148665+0.j 0.103

In [11]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [2.13305452e+02 5.79537872e+04 5.27856184e+09 1.38796234e+16
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16 3.10276249e+15 1.59813353e+16 3.10276249e+15
 1.59813353e+16

In [12]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.62014857735849+0j)
Eigenvector: [0.09296707+0.j 0.10481124+0.j 0.09983689+0.j 0.09707312+0.j
 0.10439353+0.j 0.09794769+0.j 0.10813267+0.j 0.09284344+0.j
 0.09431399+0.j 0.10806847+0.j 0.1065417 +0.j 0.10052338+0.j
 0.10371858+0.j 0.09771615+0.j 0.0998238 +0.j 0.10540399+0.j
 0.0997872 +0.j 0.10077501+0.j 0.10123154+0.j 0.09082776+0.j
 0.09291646+0.j 0.10478071+0.j 0.09539226+0.j 0.10076079+0.j
 0.10211811+0.j 0.10039803+0.j 0.10567802+0.j 0.10773907+0.j
 0.10150483+0.j 0.10974688+0.j 0.09615539+0.j 0.10295496+0.j
 0.11010317+0.j 0.0922192 +0.j 0.09679558+0.j 0.09292483+0.j
 0.10536285+0.j 0.09831243+0.j 0.09759013+0.j 0.10336412+0.j
 0.09404511+0.j 0.10061831+0.j 0.0937544 +0.j 0.10209773+0.j
 0.09780707+0.j 0.10253503+0.j 0.0990027 +0.j 0.10020791+0.j
 0.11025789+0.j 0.10165614+0.j 0.0968776 +0.j 0.10145509+0.j
 0.09507951+0.j 0.0995886 +0.j 0.09666882+0.j 0.10129013+0.j
 0.10075344+0.j 0.0963895 +0.j 0.09067848+0.j 0.10220834+0.j
 0.10205383+0.j 0.10148665+0.j 0.1039

In [13]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(49.620148577358535+0j) [-0.09296707+0.j -0.10481124+0.j -0.09983689+0.j -0.09707312+0.j
 -0.10439353+0.j -0.09794769+0.j -0.10813267+0.j -0.09284344+0.j
 -0.09431399+0.j -0.10806847+0.j -0.1065417 +0.j -0.10052338+0.j
 -0.10371858+0.j -0.09771615+0.j -0.0998238 +0.j -0.10540399+0.j
 -0.0997872 +0.j -0.10077501+0.j -0.10123154+0.j -0.09082776+0.j
 -0.09291646+0.j -0.10478071+0.j -0.09539226+0.j -0.10076079+0.j
 -0.10211811+0.j -0.10039803+0.j -0.10567802+0.j -0.10773907+0.j
 -0.10150483+0.j -0.10974688+0.j -0.09615539+0.j -0.10295496+0.j
 -0.11010317+0.j -0.0922192 +0.j -0.09679558+0.j -0.09292483+0.j
 -0.10536285+0.j -0.09831243+0.j -0.09759013+0.j -0.10336412+0.j
 -0.09404511+0.j -0.10061831+0.j -0.0937544 +0.j -0.10209773+0.j
 -0.09780707+0.j -0.10253503+0.j -0.0990027 +0.j -0.10020791+0.j
 -0.11025789+0.j -0.10165614+0.j -0.0968776 +0.j -0.10145509+0.j
 -0.09507951+0.j -0.0995886 +0.j -0.09666882+0.j -0.10129013+0.j
 -0.10075344+0.j -0.0963895 +0.j -0.09067848+0.j -0.10220834+0.j
 

In [14]:
# Function to run n rounds of RQI and RQIAPP
def runRQI(M, mu, ep, it, n):
  # Save number of iterations to arrays
  its = np.empty(0)
  itsAPP = np.empty(0)

  # Save condition numbers to arrays
  conds = np.empty(0)
  condsAPP = np.empty(0)

  # Save last condition number of each iteration to arrays
  lastConds = np.empty(0)
  lastCondsAPP = np.empty(0)

  for i in range(n):
    RQIData = RQI(M, mu, ep, it)
    RQIAPPData = RQIAPP(M, mu, ep, it)

    # Save number of iterations required to the appropriate array
    its = np.append(its, RQIData[2])
    itsAPP = np.append(itsAPP, RQIAPPData[2])

    # Save condition numbers to the appropriate array
    conds = np.append(conds, RQIData[3])
    condsAPP = np.append(condsAPP, RQIAPPData[4])

    # Save last condition number to the appropriate array
    lastConds = np.append(conds, RQIData[3][-1])
    lastCondsAPP = np.append(condsAPP, RQIAPPData[4][-1])

  # Print iterations required each round
  print('Iterations required each round for RQI:\n', its)
  print('\nIterations required each round for RQIAPP:\n', itsAPP)

  # Get average iterations required by RQI
  index = np.argwhere(its==it)
  its = np.delete(its, index)
  if its.size==0:
    itsAvg = 0
  else:
    itsAvg = np.average(its)

  # Get average condition number for RQI
  condsAvg = np.average(conds)

  # Get average final condition number for RQI
  lastCondsAvg = np.average(lastConds)

  # Get average iterations required by RQIAPP
  index = np.argwhere(itsAPP==it)
  itsAPP = np.delete(itsAPP, index)
  if itsAPP.size==0:
    itsAPPAvg = 0
  else:
    itsAPPAvg = np.average(itsAPP)

  # Get average condition number for RQIAPP
  condsAPPAvg = np.average(condsAPP)

  # Get average final condition number for RQI
  lastCondsAPPAvg = np.average(lastCondsAPP)

  # Print average iterations required and number of errors
  print('\nAverage iterations required for RQI:\n', itsAvg)
  print('\nAvergae condition number for RQI:\n', condsAvg)
  print('\nAvergae final condition number for RQI:\n', lastCondsAvg)
  print('\nPercentage of times RQI converged:\n', str(its.size/n*100) + '%')
  print('\nAverage iterations required for RQIAPP:\n', itsAPPAvg)
  print('\nAvergae condition number for RQIAPP:\n', condsAPPAvg)
  print('\nAvergae final condition number for RQIAPP:\n', lastCondsAPPAvg)
  print('\nPercentage of times RQIAPP converged:\n', str(itsAPP.size/n*100) + '%')

In [15]:
# Run 100 rounds with 1e-4
runRQI(M, mu, 1e-4, 20, 100)

Iterations required each round for RQI:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 2.0

Avergae condition number for RQI:
 47050.5258131291

Avergae final condition number for RQI:
 47022.13353208443

Percentage of times RQI converged:
 100.0%

Average iterations required for RQIAPP:
 2.0

Avergae condition number for RQIAPP:
 1.981379458560294

In [16]:
# Run 100 rounds with 1e-6
runRQI(M, mu, 1e-6, 20, 100)

Iterations required each round for RQI:
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 3.0

Avergae condition number for RQI:
 1003619610459552.2

Avergae final condition number for RQI:
 1000623732700727.0

Percentage of times RQI converged:
 98.0%

In [17]:
# Run 100 rounds with 1e-8
runRQI(M, mu, 1e-8, 20, 100)

Iterations required each round for RQI:
 [ 3.  3. 20.  3. 20.  3. 20. 20. 20. 20.  3. 20. 20. 20. 20.  3.  3. 20.
 20. 20.  3. 20. 20. 20. 20.  3. 20. 20. 20.  3.  3. 20.  3. 20. 20. 20.
 20. 20. 20.  3. 20. 20. 20. 20.  3. 20. 20.  3.  3. 20. 20. 20. 20.  3.
 20. 20.  3. 20. 20. 20. 20.  3.  3. 20. 20.  3.  3. 20. 20.  3. 20. 20.
 20. 20.  3. 20. 20. 20.  3. 20. 20. 20.  3. 20. 20. 20. 20. 20.  3. 20.
 20.  3. 20. 20.  3. 20.  3. 20. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3.
 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 3.0

Avergae condition number for RQI:
 7690225791452432.0

Avergae final condition number for RQI:
 7695786562422055.0

Percentage of times RQI converged:
 30.0%

In [18]:
# Run 100 rounds with 1e-10
runRQI(M, mu, 1e-10, 20, 100)

Iterations required each round for RQI:
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 0

Avergae condition number for RQI:
 8200004746001580.0

Avergae final condition number for RQI:
 8197457398547239.0

Percentage of times RQI converged:
 0.0%

A